In [1]:
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path
from llava.eval.run_llava import eval_model

# model_path = "liuhaotian/llava-v1.5-7b"

# tokenizer, model, image_processor, context_len = load_pretrained_model(
#     model_path=model_path,
#     model_base=None,
#     model_name=get_model_name_from_path(model_path)
# )

/datadrive_a/jihai/miniconda3/envs/llava/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-12-12 08:19:22,026] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
model_path = "/datadrive_a/jihai/LLaVA/scripts/v1_5/checkpoints/llava-v1.5-7b-lora"
prompt = "Please show me an image of 11:11:11 with red digits."
image_file = "https://llava-vl.github.io/static/images/view.jpg"

args = type('Args', (), {
    "model_path": model_path,
    "model_base": '/datadrive_a/jihai/azure_storage2/vigstandard_data/jihai/checkpoint/vicuna-7b-v1.5/vicuna-7b-v1.5',
    "model_name": get_model_name_from_path(model_path),
    "query": prompt,
    "conv_mode": None,
    "image_file": None,
    "sep": ",",
    "temperature": 0,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 512
})()

#eval_model(args)

In [3]:
import argparse
import torch
import re

import requests
from PIL import Image
from io import BytesIO

from llava.constants import (
    IMAGE_TOKEN_INDEX,
    DEFAULT_IMAGE_TOKEN,
    DEFAULT_IM_START_TOKEN,
    DEFAULT_IM_END_TOKEN,
    IMAGE_PLACEHOLDER,
)
from llava.conversation import conv_templates, SeparatorStyle
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from llava.mm_utils import (
    process_images,
    tokenizer_image_token,
    get_model_name_from_path,
)

def image_parser(args):
    out = args.image_file.split(args.sep)
    return out


def load_image(image_file):
    if image_file.startswith("http") or image_file.startswith("https"):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert("RGB")
    else:
        image = Image.open(image_file).convert("RGB")
    return image


def load_images(image_files):
    out = []
    for image_file in image_files:
        image = load_image(image_file)
        out.append(image)
    return out


model_name = get_model_name_from_path(args.model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(
    args.model_path, args.model_base, model_name
)

qs = args.query
image_token_se = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN
if IMAGE_PLACEHOLDER in qs:
    if model.config.mm_use_im_start_end:
        qs = re.sub(IMAGE_PLACEHOLDER, image_token_se, qs)
    else:
        qs = re.sub(IMAGE_PLACEHOLDER, DEFAULT_IMAGE_TOKEN, qs)
else:
    if model.config.mm_use_im_start_end:
        qs = image_token_se + "\n" + qs
    else:
        qs = DEFAULT_IMAGE_TOKEN + "\n" + qs

if "llama-2" in model_name.lower():
    conv_mode = "llava_llama_2"
elif "mistral" in model_name.lower():
    conv_mode = "mistral_instruct"
elif "v1.6-34b" in model_name.lower():
    conv_mode = "chatml_direct"
elif "v1" in model_name.lower():
    conv_mode = "llava_v1"
elif "mpt" in model_name.lower():
    conv_mode = "mpt"
else:
    conv_mode = "llava_v0"

if args.conv_mode is not None and conv_mode != args.conv_mode:
    print(
        "[WARNING] the auto inferred conversation mode is {}, while `--conv-mode` is {}, using {}".format(
            conv_mode, args.conv_mode, args.conv_mode
        )
    )
else:
    args.conv_mode = conv_mode
print(args.conv_mode)
conv = conv_templates[args.conv_mode].copy()
conv.append_message(conv.roles[0], qs)
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()

# image_files = image_parser(args)
# images = load_images(image_files)
# image_sizes = [x.size for x in images]
# images_tensor = process_images(
#     images,
#     image_processor,
#     model.config
# ).to(model.device, dtype=torch.float16)
image_sizes=None
images_tensor=None
if images_tensor is not None:
    images_tensor=images_tensor.to(model.device, dtype=torch.float16)

print(prompt)
input_ids = (
    tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt")
    .unsqueeze(0)
    .cuda()
)

Loading LLaVA from base model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]/datadrive_a/jihai/miniconda3/envs/llava/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 2/2 [02:25<00:00, 72.82s/it] 
Some weights of LlavaLlamaForCausalLM_ImgGen were not initialized from the model checkpoint at /datadrive_a/jihai/azure_storage2/vigstandard_data/jihai/checkpoint/vicuna-7b-v1.5/vicuna-7b-v1.5 and are newly initialized: ['model.mm_projector.bias', 'model.mm_projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading additional LLaVA weights...
Loading LoRA weights...
Merging LoRA weights...
Model is loaded...
llava_v1
A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions. USER: <image>
Please show me an image of 11:11:11 with red digits. ASSISTANT:


In [4]:

prompt=prompt.replace('<image>\n','')
print(prompt)
input_ids = (
    tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt")
    .unsqueeze(0)
    .cuda()
)
print(input_ids)

A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions. USER: Please show me an image of 11:11:11 with red digits. ASSISTANT:
tensor([[    1,   319, 13563,  1546,   263, 12758,  5199,   322,   385, 23116,
         21082, 20255, 29889,   450, 20255,  4076,  8444, 29892, 13173, 29892,
           322,  1248,   568,  6089,   304,   278,  5199, 29915, 29879,  5155,
         29889,  3148,  1001, 29901,  3529,  1510,   592,   385,  1967,   310,
         29871, 29896, 29896, 29901, 29896, 29896, 29901, 29896, 29896,   411,
          2654, 13340, 29889,   319,  1799,  9047, 13566, 29901]],
       device='cuda:0')


In [5]:


with torch.inference_mode():
    outputs = model.generate(
        input_ids,
        images=images_tensor,
        image_sizes=image_sizes,
        do_sample=True if args.temperature > 0 else False,
        temperature=args.temperature,
        top_p=args.top_p,
        num_beams=args.num_beams,
        max_new_tokens=args.max_new_tokens,
        use_cache=True,
    )

output_ids=outputs['generated_tokens']
hiddens=outputs['hidden_states']


outputs = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0].strip()

# # 解码生成的 token
# if tokenizer is not None:
#     generated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=False)
# else:
#     generated_text = generated_tokens[0].tolist()  # 直接返回 token IDs

# # 检查生成文本中是否包含 `<image>`
# image_start_idx = None
# image_end_idx = None
# if tokenizer is not None:
#     tokens = tokenizer.convert_ids_to_tokens(generated_tokens[0])
#     if "<image>" in tokens:
#         image_start_idx = tokens.index("<image>")
#         # 查找终止符号（假设为 `<eos>` 或句子结束符号）
#         image_end_idx = len(tokens)
#         for i, token in enumerate(tokens[image_start_idx + 1:], start=image_start_idx + 1):
#             if token in ["<eos>", ".", "!", "?"]:  # 自定义终止符号
#                 image_end_idx = i
#                 break

# # 提取 `<image>` 到终止符号之间的 hidden states
# image_hidden_states = None
# if image_start_idx is not None and image_end_idx is not None:
#     image_hidden_states = hidden_states[:, image_start_idx:image_end_idx, :]
print(outputs)

inputs_embeds: tensor([[[ 0.0027, -0.0041,  0.0021,  ..., -0.0098,  0.0035, -0.0041],
         [-0.0098, -0.0132, -0.0127,  ...,  0.0089,  0.0129, -0.0085],
         [ 0.0218, -0.0081,  0.0075,  ...,  0.0138, -0.0019, -0.0224],
         ...,
         [-0.0179, -0.0002,  0.0173,  ...,  0.0246,  0.0054,  0.0096],
         [ 0.0063, -0.0166,  0.0113,  ..., -0.0105,  0.0144,  0.0071],
         [ 0.0032,  0.0017,  0.0056,  ..., -0.0054,  0.0162,  0.0022]]],
       device='cuda:0', dtype=torch.float16)
inputs_embeds: None
inputs_embeds: None


/datadrive_a/jihai/miniconda3/envs/llava/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/datadrive_a/jihai/miniconda3/envs/llava/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


inputs_embeds: None


inputs_embeds: None
inputs_embeds: None
inputs_embeds: None
inputs_embeds: None
inputs_embeds: None
inputs_embeds: None
inputs_embeds: None
inputs_embeds: None
inputs_embeds: None
inputs_embeds: None
inputs_embeds: None
inputs_embeds: None
inputs_embeds: None
inputs_embeds: None
inputs_embeds: None
inputs_embeds: None
inputs_embeds: None
inputs_embeds: None
inputs_embeds: None
inputs_embeds: None
inputs_embeds: None
inputs_embeds: None
inputs_embeds: None
inputs_embeds: None
inputs_embeds: None
inputs_embeds: None
outputs: GenerateDecoderOnlyOutput(sequences=tensor([[    1,   910,   338,   278,  1967,   310, 29871, 29896, 29896, 29901,
         29896, 29896, 29901, 29896, 29896,   411,  2654, 13340, 29889, 29871,
            13,   529,  3027, 29958, 14899, 29891, 29889, 29871,   486, 29878,
             2]], device='cuda:0'), scores=None, attentions=None, hidden_states=((tensor([[[ 0.0027, -0.0041,  0.0021,  ..., -0.0098,  0.0035, -0.0041],
         [-0.0098, -0.0132, -0.0127,  ...,  0

In [ ]:
print(input_ids.shape)
print(new_input_embeds.shape)

In [ ]:
import torch
a=torch.tensor([1,2,3])
b=torch.LongTensor([1,1,0]).bool()
print(a[b])

In [ ]:
import numpy as np

# 1. 生成一个随机的可逆矩阵 A
def generate_invertible_matrix(dim):
    # 随机生成一个矩阵
    random_matrix = np.random.randn(dim, dim)
    
    # 通过 QR 分解确保矩阵 A 是可逆的（正交矩阵 Q 总是可逆的）
    Q, _ = np.linalg.qr(random_matrix)  # QR 分解，Q 是正交矩阵
    return Q

# 2. 生成一个随机的平移向量 b
def generate_translation_vector(dim):
    return np.random.randn(dim)

# 3. 仿射变换函数 y = A * x + b
def affine_transform(x, A, b):
    return np.dot(A, x) + b

# 4. 逆仿射变换 x = A^(-1) * (y - b)
def inverse_affine_transform(y, A, b):
    A_inv = np.linalg.inv(A)
    return np.dot(A_inv, (y - b))

# 维度 4096
dim = 4096

# 随机生成可逆矩阵 A 和平移向量 b
A = generate_invertible_matrix(dim)
b = generate_translation_vector(dim)

# 随机生成 4096 维输入向量 x
x = np.random.randn(dim)

# 进行仿射变换 y = A * x + b
y = affine_transform(x, A, b)

# 进行逆仿射变换，检查是否能恢复原始的 x
x_reconstructed = inverse_affine_transform(y, A, b)

# 打印重构误差，检查是否正确恢复
reconstruction_error = np.linalg.norm(x - x_reconstructed)
print("Reconstruction error:", reconstruction_error)
